New Code

In [ ]:
import os
import zipfile
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from vit_pytorch import ViT
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error
import pandas as pd
from tqdm import tqdm
from torch.utils.data import DataLoader
from pytorch_lamb import Lamb
from torch.optim import AdamW
import timm 
from sklearn.metrics import classification_report, confusion_matrix
#unprocessed Data
# dataset_path = '../skinType' 

#preprocessed Data
dataset_path = '../Preprocessing/Processed_Images'
#dataset_path="../skintypepatches 128x128"
#_FaceCrops_256' 

# Define hyperparameters
batch_sizes = [ 8, 32,64]
learning_rates =[0.01, 0.001]
optimizers_list = ['LAMB', 'AdamW', 'SGD', 'RAdam']

num_classes = 3  # Dry, Normal, Oily skin types
#class_names = ['oily', 'dry', 'normal']
steps_per_epoch = 20 
total_epochs=500

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
# Load Dataset
transform = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])
# train_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(10),
#     transforms.ToTensor(),
#     transforms.Normalize(mean, std)
# ])

# val_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean, std)
# ])
# Dataset Path
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)


train_size = int(0.7 * len(dataset))
val_size = int(0.20 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

#val_dataset.dataset.transform = val_transform
print(f"train size {train_size}")
print(f"Val size {val_size}")
print(f"Test Size {test_size}")

# print(f"train Dataset {train_dataset}")
# print(f"Val Dataset {val_dataset}")
# print(f"Test Dataset {test_dataset}")

# Function to initialize the model
# def create_vit_model():    
#     model = ViT(
#         image_size=128,
#         patch_size=16,
#         num_classes=num_classes,  
#         dim=512,                      
#         heads=8,  
#         depth=8,              
#         mlp_dim=512,          
#         dropout=0.4,           
#         emb_dropout=0.4        
#     )
    
#     return model

# Optimizer choices
def get_optimizer(optimizer_name, model_params, lr,weight_decay=0.001):
    if optimizer_name == 'AdamW':
        return optim.AdamW(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'LAMB':
        return Lamb(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'SGD':
        return SGD(model_params, lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name == 'RAdam':
        return RAdam(model_params, lr=lr, weight_decay=weight_decay)

# Training Function
def train_model(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []
    y_true, y_pred = [], []
    all_probs = []

    for inputs, labels in tqdm(loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.detach().cpu().numpy())
        
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

    try:
        unique_classes = np.unique(all_labels)
        if len(unique_classes) == num_classes:
            roc_auc = roc_auc_score(all_labels, np.array(all_probs), multi_class='ovr', average='weighted')
        else:
            roc_auc = None
    except ValueError:
        roc_auc = None 

    rmse = np.sqrt(mean_squared_error(all_labels, all_preds))

    return running_loss / len(loader), accuracy, precision, recall, f1, roc_auc, rmse

# Validation and Test Function
def evaluate_model(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []
    y_true, y_pred = [], []
    
    all_probs = [] 

    with torch.no_grad():
        for inputs, labels in tqdm(loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)
            
            loss = criterion(outputs, labels)
            running_loss += loss.item()


            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    
    try:
       
        unique_classes = np.unique(all_labels)
        if len(unique_classes) == num_classes:
            roc_auc = roc_auc_score(all_labels, np.array(all_probs), multi_class='ovr', average='weighted')
        else:
            print("Warning: Not all classes are present in the evaluation set. Skipping ROC-AUC calculation.")
            roc_auc = None
    except ValueError:
        roc_auc = None  

    rmse = np.sqrt(mean_squared_error(all_labels, all_preds))
    
    return running_loss / len(loader), accuracy, precision, recall, f1, roc_auc, rmse


# Main training loop
def train_and_evaluate(batch_size, lr, optimizer_name):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Using device:', device)

    # Dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)
    model.to(device)
    optimizer = get_optimizer(optimizer_name, model.parameters(), lr)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    # Training
    for epoch in range(100,total_epochs,20):
        # print("\n\n-------------------------Checking Weights in Each Iteration----------------------------")
        # print("Initial Weights:")
        # print_weights(model)
        # print("\n\n-------------------------Checking Weights in Each Iteration-----------------------------")
  
        train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = train_model(model, train_loader, optimizer, criterion, device)
        
        val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse= evaluate_model(model, val_loader, criterion, device)
        test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = evaluate_model(model, test_loader, criterion, device)
        
        scheduler.step()
        scheduler.step(val_loss)
        print("----------Values After Training-----------")
        print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}",
              f"\nOptimizer: {optimizer_name} \nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                   f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train  F1: {train_f1:.4f},"
                   f"Train ROC AUC: {train_roc_auc:.4f}, Train RMSE: {train_rmse:.4f}")
        
        print("\n\n-----------Values After Validation-----------")
        print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
              f"\nOptimizer: {optimizer_name} \nVal Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                    f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc:.4f}, "
                    f"Val RMSE: {val_rmse:.4f}")
        
        print("\n\n-----------Values After Testing-----------")
        print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
              f"\nOptimizer: {optimizer_name} Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Precision: {test_precision:.4f}, "
                    f"Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}, Test RMSE: {test_rmse:.4f}")
                
        
       
        
        #Saving Training data
        overall_result = {
            'Epoch': epoch + 1,
            'Batch Size': batch_size,
            'Learning Rate': lr,
            'Optimizer': optimizer_name,
            
            'Train Loss':round(train_loss, 4),
            'Test Loss':round(test_loss, 4),
            'Val Loss':round(val_loss, 4),
            
            'Train Acc': round(train_acc, 4),
            'Test Acc': round(test_acc, 4),
            'Val Acc': round(val_acc, 4),
            
            
            'Train Precision': round(train_precision, 4),
            'Test Precision': round(test_precision, 4),
            'Val Precision': round(val_precision, 4),
            
            'Train Recall': round(train_recall, 4),
            'Test Recall': round(test_recall, 4),
            'Val Recall': round(val_recall, 4),
            
            'Train F1 Score': round(train_f1, 4),
            'Test F1 Score': round(test_f1, 4),
            'Val F1 Score': round(val_f1, 4),
            
            'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
            'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
            'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,
            
            'Train RMSE': round(train_rmse, 4),
            'Test RMSE': round(test_rmse, 4),
            'Val RMSE': round(val_rmse, 4)
        }
        
        # Append to CSV
        overall_result_file = 'pretrained_overall_result.csv'
        
        if not os.path.isfile(overall_result_file):
            pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)        
        else:
            pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)
            
    # Save the trained model
    os.makedirs('../saved_models', exist_ok=True)
    model_save_path = f"../saved_models/vit_model_bs{batch_size}_lr{lr}_optimizer{optimizer_name}.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")

    # # Convert results into DataFrame
    df_results = pd.read_csv('pretrained_overall_result.csv')

    # Reshape the dataframe to long format for seaborn
    df_long = pd.melt(df_results, id_vars=['Epoch'], var_name='Metric', value_name='Value')

    # Optional: Drop rows with NaN values in the Value column (if any)
    df_long = df_long.dropna(subset=['Value'])

    # Set up the plots with a style
    sns.set_theme(style="whitegrid")

    # Plot Loss (Train, Test, Val)
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_long[df_long['Metric'].isin(['Train Loss', 'Test Loss', 'Val Loss'])],x='Epoch', y='Value', hue='Metric')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(title='Metric')
    plt.show()

    # Plot Accuracy (Train, Test, Val)
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_long[df_long['Metric'].isin(['Train Acc', 'Test Acc', 'Val Acc'])],
                x='Epoch', y='Value', hue='Metric')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(title='Metric')
    plt.show()

    # Plot Precision (Train, Test, Val)
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_long[df_long['Metric'].isin(['Train Precision', 'Test Precision', 'Val Precision'])],x='Epoch', y='Value', 
                 hue='Metric')
    plt.title('Precision Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Precision')
    plt.legend(title='Metric')
    plt.show()

    # Plot Recall (Train, Test, Val)
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_long[df_long['Metric'].isin(['Train Recall', 'Test Recall', 'Val Recall'])],x='Epoch', y='Value', hue='Metric')
    plt.title('Recall Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.legend(title='Metric')
    plt.show()

    # Plot F1 Score (Train, Test, Val)
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_long[df_long['Metric'].isin(['Train F1 Score', 'Test F1 Score', 'Val F1 Score'])],
                x='Epoch', y='Value', hue='Metric')
    plt.title('F1 Score Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend(title='Metric')
    plt.show()

    # Plot ROC AUC (Train, Test, Val)
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_long[df_long['Metric'].isin(['Train ROC AUC', 'Test ROC AUC', 'Val ROC AUC'])],
                x='Epoch', y='Value', hue='Metric')
    plt.title('ROC AUC Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('ROC AUC')
    plt.legend(title='Metric')
    plt.show()

    # Plot RMSE (Train, Test, Val)
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_long[df_long['Metric'].isin(['Train RMSE', 'Test RMSE', 'Val RMSE'])],
                x='Epoch', y='Value', hue='Metric')
    plt.title('RMSE Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('RMSE')
    plt.legend(title='Metric')
    plt.show()

    # return test_acc, precision, recall, f1, roc_auc, rmse

for batch_size in batch_sizes:
    for lr in learning_rates:
        for optimizer_name in optimizers_list:
                #test_metrics = 
                train_and_evaluate(batch_size, lr, optimizer_name)
                # print(f"Final Test Metrics with Batch Size {batch_size}, LR {lr}, Optimizer {optimizer_name}: {test_metrics}")
                
            

New Model With Clipping

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
from torch_optimizer import Lamb
import clip

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Hyperparameters
batch_sizes = [8]
learning_rates = [0.01, 0.001]
optimizers_list = ['LAMB', 'AdamW']

# batch_sizes = [8, 32, 64]
# learning_rates = [0.01, 0.001]
# optimizers_list = ['LAMB', 'AdamW']

total_epochs = 250
start=1
step=1

# Load CLIP model and preprocessing
clip_model, _ = clip.load("ViT-B/32", device=device)

# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])

# Dataset
data_dir = r"../skintypepatches 128x128"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print("Classes:", dataset.classes)

# Data Splitting
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


print(f"train size {train_size}")
print(f"Val size {val_size}")
print(f"Test Size {test_size}")

def get_optimizer(optimizer_name, model_params, lr,weight_decay=0.001):
    if optimizer_name == 'AdamW':
        return optim.AdamW(model_params, lr=lr,weight_decay=weight_decay)
    elif optimizer_name=='LAMB':
        return Lamb(model_params, lr=lr,weight_decay=weight_decay)
    
    
# Freeze CLIP vision encoder
for param in clip_model.visual.parameters():
    param.requires_grad = False

# Model definition
class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=3):
        super(CLIPSkinClassifier, self).__init__()
        self.encoder = clip_model.visual
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.encoder(x)
        logits = self.classifier(features)
        return logits

model = CLIPSkinClassifier(clip_model, num_classes=3).to(device).float()



# Metric helper
def compute_metrics(outputs, labels):
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)
    try:
        roc_auc = roc_auc_score(np.eye(3)[labels], F.softmax(outputs, dim=1).cpu().detach().numpy(), multi_class='ovr')
    except:
        roc_auc = None
    rmse = root_mean_squared_error(labels, preds)
    return acc, precision, recall, f1, roc_auc, rmse

# Train loop
def train_model(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_outputs = []
    all_labels = []

    for images, labels in tqdm(loader):
        images, labels = images.to(device).float(), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        all_outputs.append(outputs.detach())
        all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse

# Eval loop
def evaluate_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(loader):
            images, labels = images.to(device).float(), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            all_outputs.append(outputs)
            all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse

# Training loop with logging



for batch_size in batch_sizes:
    for lr in learning_rates:
        for optimizer_name in optimizers_list:
            
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
            
            # Loss and optimizer
            criterion = nn.CrossEntropyLoss()
            optimizer = Lamb(model.parameters(), lr=lr)
            
            for epoch in range(start,total_epochs,step):
                train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = train_model(model, train_loader, optimizer, criterion, device)
                val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse = evaluate_model(model, val_loader, criterion, device)
                test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = evaluate_model(model, test_loader, criterion, device)

                print("----------Values After Training-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}",
                    f"\nOptimizer: {optimizer_name} \nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                        f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train  F1: {train_f1:.4f},"
                        f"Train ROC AUC: {train_roc_auc:.4f}, Train RMSE: {train_rmse:.4f}")

                print("\n\n-----------Values After Validation-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name} \nVal Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                            f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc:.4f}, "
                            f"Val RMSE: {val_rmse:.4f}")

                print("\n\n-----------Values After Testing-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name} Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Precision: {test_precision:.4f}, "
                            f"Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}, Test RMSE: {test_rmse:.4f}")

                # Save results to CSV
                overall_result = {
                    'Epoch': epoch + 1,
                    'Batch Size': batch_size,
                    'Learning Rate': lr,
                    'Optimizer': optimizer_name,

                    'Train Loss': round(train_loss, 4),
                    'Test Loss': round(test_loss, 4),
                    'Val Loss': round(val_loss, 4),

                    'Train Acc': round(train_acc, 4),
                    'Test Acc': round(test_acc, 4),
                    'Val Acc': round(val_acc, 4),

                    'Train Precision': round(train_precision, 4),
                    'Test Precision': round(test_precision, 4),
                    'Val Precision': round(val_precision, 4),

                    'Train Recall': round(train_recall, 4),
                    'Test Recall': round(test_recall, 4),
                    'Val Recall': round(val_recall, 4),

                    'Train F1 Score': round(train_f1, 4),
                    'Test F1 Score': round(test_f1, 4),
                    'Val F1 Score': round(val_f1, 4),

                    'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
                    'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
                    'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,

                    'Train RMSE': round(train_rmse, 4),
                    'Test RMSE': round(test_rmse, 4),
                    'Val RMSE': round(val_rmse, 4)
                }

                overall_result_file = 'sample result (250).csv'
                if not os.path.isfile(overall_result_file):
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)
                else:
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)

            # Save model
            os.makedirs('../saved_models', exist_ok=True)
            model_save_path = f"../saved_models/vit_model_bs{batch_size}_lr{lr}_optimizer{optimizer_name}.pth"
            torch.save(model.state_dict(), model_save_path)
            print(f"Model saved to {model_save_path}")




In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
from torch_optimizer import Lamb
import clip

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Hyperparameters
batch_sizes = [8]
learning_rates = [0.001]
optimizers_list = ['LAMB', 'AdamW']
total_epochs = 500
start=100
step=20

# Load CLIP model and preprocessing
clip_model, _ = clip.load("ViT-B/32", device=device)

# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])

# Dataset
data_dir = r"../skintypepatches 128x128"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print("Classes:", dataset.classes)

# Data Splitting
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

print(f"train size {train_size}")
print(f"Val size {val_size}")
print(f"Test Size {test_size}")


def get_optimizer(optimizer_name, model_params, lr,weight_decay=0.001):
    if optimizer_name == 'AdamW':
        return optim.AdamW(model_params, lr=lr,weight_decay=weight_decay)
    elif optimizer_name=='LAMB':
        return Lamb(model_params, lr=lr,weight_decay=weight_decay)
    
    
# Freeze CLIP vision encoder
for param in clip_model.visual.parameters():
    param.requires_grad = False

# Model definition
class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=3):
        super(CLIPSkinClassifier, self).__init__()
        self.encoder = clip_model.visual
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.encoder(x)
        logits = self.classifier(features)
        return logits

model = CLIPSkinClassifier(clip_model, num_classes=3).to(device).float()



# Metric helper
def compute_metrics(outputs, labels):
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)
    try:
        roc_auc = roc_auc_score(np.eye(3)[labels], F.softmax(outputs, dim=1).cpu().detach().numpy(), multi_class='ovr')
    except:
        roc_auc = None
    rmse = root_mean_squared_error(labels, preds)
    return acc, precision, recall, f1, roc_auc, rmse

# Train loop
def train_model(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_outputs = []
    all_labels = []

    for images, labels in tqdm(loader):
        images, labels = images.to(device).float(), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        all_outputs.append(outputs.detach())
        all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse

# Eval loop
def evaluate_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(loader):
            images, labels = images.to(device).float(), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            all_outputs.append(outputs)
            all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse

# Training loop with logging
for batch_size in batch_sizes:
    for lr in learning_rates:
        for optimizer_name in optimizers_list:
            
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
            
            # Loss and optimizer
            criterion = nn.CrossEntropyLoss()
            optimizer = Lamb(model.parameters(), lr=lr)
            
            for epoch in range(start,total_epochs,step):
                train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = train_model(model, train_loader, optimizer, criterion, device)
                val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse = evaluate_model(model, val_loader, criterion, device)
                test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = evaluate_model(model, test_loader, criterion, device)

                print("----------Values After Training-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}",
                    f"\nOptimizer: {optimizer_name} \nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                        f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train  F1: {train_f1:.4f},"
                        f"Train ROC AUC: {train_roc_auc:.4f}, Train RMSE: {train_rmse:.4f}")

                print("\n\n-----------Values After Validation-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name} \nVal Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                            f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc:.4f}, "
                            f"Val RMSE: {val_rmse:.4f}")

                print("\n\n-----------Values After Testing-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name} Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Precision: {test_precision:.4f}, "
                            f"Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}, Test RMSE: {test_rmse:.4f}")

                # Save results to CSV
                overall_result = {
                    'Epoch': epoch + 1,
                    'Batch Size': batch_size,
                    'Learning Rate': lr,
                    'Optimizer': optimizer_name,

                    'Train Loss': round(train_loss, 4),
                    'Test Loss': round(test_loss, 4),
                    'Val Loss': round(val_loss, 4),

                    'Train Acc': round(train_acc, 4),
                    'Test Acc': round(test_acc, 4),
                    'Val Acc': round(val_acc, 4),

                    'Train Precision': round(train_precision, 4),
                    'Test Precision': round(test_precision, 4),
                    'Val Precision': round(val_precision, 4),

                    'Train Recall': round(train_recall, 4),
                    'Test Recall': round(test_recall, 4),
                    'Val Recall': round(val_recall, 4),

                    'Train F1 Score': round(train_f1, 4),
                    'Test F1 Score': round(test_f1, 4),
                    'Val F1 Score': round(val_f1, 4),

                    'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
                    'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
                    'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,

                    'Train RMSE': round(train_rmse, 4),
                    'Test RMSE': round(test_rmse, 4),
                    'Val RMSE': round(val_rmse, 4)
                }

                overall_result_file = 'sample result (250).csv'
                if not os.path.isfile(overall_result_file):
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)
                else:
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)

            # Save model
            os.makedirs('../saved_models', exist_ok=True)
            model_save_path = f"../saved_models/vit_model_bs{batch_size}_lr{lr}_optimizer{optimizer_name}.pth"
            torch.save(model.state_dict(), model_save_path)
            print(f"Model saved to {model_save_path}")



In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
from torch_optimizer import Lamb  # You may need to install this via pip install pytorch-optimizer
import clip
from torch.optim import AdamW

# Device setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Hyperparameters
batch_size = 8
lr = 0.0001
optimizer_name = "AdamW"
total_epochs = 501
start=1
step=1

# Load CLIP model and preprocessing
clip_model, _ = clip.load("ViT-B/32", device=device)

# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])

# Dataset
data_dir = "../skintypepatches 128x128"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print("Classes:", dataset.classes)

# Data Splitting
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Freeze CLIP vision encoder
for param in clip_model.visual.parameters():
    param.requires_grad = False

# Model definition
class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=3):
        super(CLIPSkinClassifier, self).__init__()
        self.encoder = clip_model.visual
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.encoder(x)
        logits = self.classifier(features)
        return logits

model = CLIPSkinClassifier(clip_model, num_classes=3).to(device).float()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
#optimizer = Lamb(model.parameters(), lr=lr)
optimizer = AdamW(model.parameters(), lr=lr, weight_decay=0.001)
# Metric helper
def compute_metrics(outputs, labels):
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)
    try:
        roc_auc = roc_auc_score(np.eye(3)[labels], F.softmax(outputs, dim=1).cpu().detach().numpy(), multi_class='ovr')
    except:
        roc_auc = None
    rmse = root_mean_squared_error(labels, preds)
    return acc, precision, recall, f1, roc_auc, rmse

# Train loop
def train_model(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_outputs = []
    all_labels = []

    for images, labels in tqdm(loader):
        images, labels = images.to(device).float(), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        all_outputs.append(outputs.detach())
        all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse

# Eval loop
def evaluate_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(loader):
            images, labels = images.to(device).float(), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            all_outputs.append(outputs)
            all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse

# Training loop with logging
for epoch in range(start,total_epochs,step):
    train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = train_model(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse = evaluate_model(model, val_loader, criterion, device)
    test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = evaluate_model(model, test_loader, criterion, device)

    print("----------Values After Training-----------")
    print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}",
          f"\nOptimizer: {optimizer_name} \nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
               f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train  F1: {train_f1:.4f},"
               f"Train ROC AUC: {train_roc_auc:.4f}, Train RMSE: {train_rmse:.4f}")

    print("\n\n-----------Values After Validation-----------")
    print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
          f"\nOptimizer: {optimizer_name} \nVal Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc:.4f}, "
                f"Val RMSE: {val_rmse:.4f}")

    print("\n\n-----------Values After Testing-----------")
    print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
          f"\nOptimizer: {optimizer_name} Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Precision: {test_precision:.4f}, "
                f"Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}, Test RMSE: {test_rmse:.4f}")

    # Save results to CSV
    overall_result = {
        'Epoch': epoch + 1,
        'Batch Size': batch_size,
        'Learning Rate': lr,
        'Optimizer': optimizer_name,

        'Train Loss': round(train_loss, 4),
        'Test Loss': round(test_loss, 4),
        'Val Loss': round(val_loss, 4),

        'Train Acc': round(train_acc, 4),
        'Test Acc': round(test_acc, 4),
        'Val Acc': round(val_acc, 4),

        'Train Precision': round(train_precision, 4),
        'Test Precision': round(test_precision, 4),
        'Val Precision': round(val_precision, 4),

        'Train Recall': round(train_recall, 4),
        'Test Recall': round(test_recall, 4),
        'Val Recall': round(val_recall, 4),

        'Train F1 Score': round(train_f1, 4),
        'Test F1 Score': round(test_f1, 4),
        'Val F1 Score': round(val_f1, 4),

        'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
        'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
        'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,

        'Train RMSE': round(train_rmse, 4),
        'Test RMSE': round(test_rmse, 4),
        'Val RMSE': round(val_rmse, 4)
    }

    overall_result_file = 'test with shuffle.csv'
    if not os.path.isfile(overall_result_file):
        pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)
    else:
        pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)

Final testing

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
from torch_optimizer import Lamb  
import clip
from torch.optim import AdamW
from torch.optim import SGD
from pytorch_lamb import Lamb
from torch_optimizer import RAdam


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")



batch_sizes = [8,16,32,64]
learning_rates = [0.1,0.01,0.001,0.0001,0.00001]
optimizers_list = ['LAMB', 'AdamW', 'SGD', 'RAdam']
num_classes=3
total_epochs = 2
start=1
step=1



clip_model, _ = clip.load("ViT-B/32", device=device)



transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])



data_dir = "../skintypepatches 128x128"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print("Classes:", dataset.classes)



train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


for param in clip_model.visual.parameters():
    param.requires_grad = False


class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=3):
        super(CLIPSkinClassifier, self).__init__()
        self.encoder = clip_model.visual
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.encoder(x)
        logits = self.classifier(features)
        return logits



criterion = nn.CrossEntropyLoss()
def get_optimizer(optimizer_name, model_params, lr,weight_decay=0.001):
    if optimizer_name == 'AdamW':
        return AdamW(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'LAMB':
        return Lamb(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'SGD':
        return SGD(model_params, lr=lr, momentum=0.9, weight_decay=weight_decay)
    elif optimizer_name == 'RAdam':
        return RAdam(model_params, lr=lr, weight_decay=weight_decay)


def compute_metrics(outputs, labels):
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)
    try:
        roc_auc = roc_auc_score(np.eye(3)[labels], F.softmax(outputs, dim=1).cpu().detach().numpy(), multi_class='ovr')
    except:
        roc_auc = None
    rmse = root_mean_squared_error(labels, preds)
    return acc, precision, recall, f1, roc_auc, rmse



def train_model(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_outputs = []
    all_labels = []

    for images, labels in tqdm(loader):
        images, labels = images.to(device).float(), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        all_outputs.append(outputs.detach())
        all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse



def evaluate_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(loader):
            images, labels = images.to(device).float(), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            all_outputs.append(outputs)
            all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse




for batch_size in batch_sizes:
    for lr in learning_rates:
        for optimizer_name in optimizers_list:
            
            model = CLIPSkinClassifier(clip_model, num_classes=num_classes).to(device).float()
            
            
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
            
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
            
            
            criterion = nn.CrossEntropyLoss()
            optimizer = get_optimizer(optimizer_name,model.parameters(), lr=lr)
            
            
            for epoch in range(start,total_epochs,step):
                
                
                train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = train_model(model, train_loader, optimizer, criterion, device)
                val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse = evaluate_model(model, val_loader, criterion, device)
                test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = evaluate_model(model, test_loader, criterion, device)


                print("----------Values After Training-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}",
                    f"\nOptimizer: {optimizer_name} \nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                        f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train  F1: {train_f1:.4f},"
                        f"Train ROC AUC: {train_roc_auc if train_roc_auc is not None else 'N/A'}, Train RMSE: {train_rmse:.4f}")

                print("\n\n-----------Values After Validation-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name} \nVal Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                            f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc if train_roc_auc is not None else 'N/A'}, "
                            f"Val RMSE: {val_rmse:.4f}")

                print("\n\n-----------Values After Testing-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name} Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Precision: {test_precision:.4f}, "
                            f"Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}, Test ROC AUC: {test_roc_auc if train_roc_auc is not None else 'N/A'}, Test RMSE: {test_rmse:.4f}")

                
                
                overall_result = {
                    'Epoch': epoch + 1,
                    'Batch Size': batch_size,
                    'Learning Rate': lr,
                    'Optimizer': optimizer_name,

                    'Train Loss': round(train_loss, 4),
                    'Test Loss': round(test_loss, 4),
                    'Val Loss': round(val_loss, 4),

                    'Train Acc': round(train_acc, 4),
                    'Test Acc': round(test_acc, 4),
                    'Val Acc': round(val_acc, 4),

                    'Train Precision': round(train_precision, 4),
                    'Test Precision': round(test_precision, 4),
                    'Val Precision': round(val_precision, 4),

                    'Train Recall': round(train_recall, 4),
                    'Test Recall': round(test_recall, 4),
                    'Val Recall': round(val_recall, 4),

                    'Train F1 Score': round(train_f1, 4),
                    'Test F1 Score': round(test_f1, 4),
                    'Val F1 Score': round(val_f1, 4),

                    'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
                    'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
                    'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,

                    'Train RMSE': round(train_rmse, 4),
                    'Test RMSE': round(test_rmse, 4),
                    'Val RMSE': round(val_rmse, 4)
                }

                overall_result_file = 'final testing.csv'
                if not os.path.isfile(overall_result_file):
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)
                else:
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)

My code

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
from torch_optimizer import Lamb  
import clip
from torch.optim import AdamW
from torch.optim import SGD
from pytorch_lamb import Lamb
from torch_optimizer import RAdam


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")



batch_sizes = [8]
learning_rates = [0.001]
optimizers_list = ['AdamW','RAdam']
num_classes=3
total_epochs =501
start=100
step=5



clip_model, _ = clip.load("ViT-B/32", device=device)



transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])



data_dir = "../Preprocessing/skintypepatches 128x128"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print("Classes:", dataset.classes)



train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


for param in clip_model.visual.parameters():
    param.requires_grad = False


class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=3):
        super(CLIPSkinClassifier, self).__init__()
        self.encoder = clip_model.visual
        self.classifier = nn.Sequential(
            nn.Linear(self.encoder.output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        features = self.encoder(x)
        logits = self.classifier(features)
        return logits





criterion = nn.CrossEntropyLoss()
def get_optimizer(optimizer_name, model_params, lr,weight_decay=0.001):
    if optimizer_name == 'AdamW':
        return AdamW(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'LAMB':
        return Lamb(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'SGD':
        return SGD(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'RAdam':
        return RAdam(model_params, lr=lr, weight_decay=weight_decay)


def compute_metrics(outputs, labels):
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)
    try:
        roc_auc = roc_auc_score(np.eye(3)[labels], F.softmax(outputs, dim=1).cpu().detach().numpy(), multi_class='ovr')
    except:
        roc_auc = None
    rmse = root_mean_squared_error(labels, preds)
    return acc, precision, recall, f1, roc_auc, rmse



def train_model(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_outputs = []
    all_labels = []

    for images, labels in tqdm(loader):
        images, labels = images.to(device).float(), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        all_outputs.append(outputs.detach())
        all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse



def evaluate_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(loader):
            images, labels = images.to(device).float(), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            all_outputs.append(outputs)
            all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse




for batch_size in batch_sizes:
    for lr in learning_rates:
        for optimizer_name in optimizers_list:
            
            model = CLIPSkinClassifier(clip_model, num_classes=num_classes).to(device).float()
            
            
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
            
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
            
            
            criterion = nn.CrossEntropyLoss()
            optimizer = get_optimizer(optimizer_name,model.parameters(), lr=lr)
            
            
            for epoch in range(start,total_epochs,step):
                
                
                train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = train_model(model, train_loader, optimizer, criterion, device)
                val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse = evaluate_model(model, val_loader, criterion, device)
                test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = evaluate_model(model, test_loader, criterion, device)


                print("----------Values After Training-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}",
                    f"\nOptimizer: {optimizer_name} \nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                        f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train  F1: {train_f1:.4f},"
                        f"Train ROC AUC: {train_roc_auc if train_roc_auc is not None else 'N/A'}, Train RMSE: {train_rmse:.4f}")

                print("\n\n-----------Values After Validation-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name} \nVal Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                            f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc if train_roc_auc is not None else 'N/A'}, "
                            f"Val RMSE: {val_rmse:.4f}")

                print("\n\n-----------Values After Testing-----------")
                print(f"\nEpoch: [{epoch+1}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name} Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Precision: {test_precision:.4f}, "
                            f"Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}, Test ROC AUC: {test_roc_auc if train_roc_auc is not None else 'N/A'}, Test RMSE: {test_rmse:.4f}")

                
                
                overall_result = {
                    'Epoch': epoch ,
                    'Batch Size': batch_size,
                    'Learning Rate': lr,
                    'Optimizer': optimizer_name,

                    'Train Loss': round(train_loss, 4),
                    'Test Loss': round(test_loss, 4),
                    'Val Loss': round(val_loss, 4),

                    'Train Acc': round(train_acc, 4),
                    'Test Acc': round(test_acc, 4),
                    'Val Acc': round(val_acc, 4),

                    'Train Precision': round(train_precision, 4),
                    'Test Precision': round(test_precision, 4),
                    'Val Precision': round(val_precision, 4),

                    'Train Recall': round(train_recall, 4),
                    'Test Recall': round(test_recall, 4),
                    'Val Recall': round(val_recall, 4),

                    'Train F1 Score': round(train_f1, 4),
                    'Test F1 Score': round(test_f1, 4),
                    'Val F1 Score': round(val_f1, 4),

                    'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
                    'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
                    'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,

                    'Train RMSE': round(train_rmse, 4),
                    'Test RMSE': round(test_rmse, 4),
                    'Val RMSE': round(val_rmse, 4)
                }

                overall_result_file = 'special final testing.csv'
                if not os.path.isfile(overall_result_file):
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)
                else:
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)
                    
        

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
from torch_optimizer import Lamb  
import clip
from torch.optim import AdamW
from torch.optim import SGD
from pytorch_lamb import Lamb
from torch_optimizer import RAdam
from sklearn.model_selection import KFold
from torch.utils.data import Subset

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")



batch_sizes = [4,8,16,32,64]
#[4,8,16,32,64]
learning_rates = [0.1,0.01,0.001,0.0001,0.00001]
#[0.1,0.01,0.001,0.0001,0.00001]
optimizers_list = ['LAMB', 'AdamW', 'SGD', 'RAdam']
#['LAMB', 'AdamW', 'SGD', 'RAdam']
num_classes=3
total_epochs = 501
start=100
step=5



clip_model, _ = clip.load("ViT-B/32", device=device)


#without augmentataion
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])

#with augmentaion
# aug_transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(p=0.8),
#     transforms.RandomVerticalFlip(p=0.8),
#     transforms.RandomRotation(45),     
#     transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
#     #transforms.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(0.75, 1.25)),  
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
#                          (0.26862954, 0.26130258, 0.27577711))
# ])

data_dir = "../Preprocessing/stage3patches"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
print("Classes:", dataset.classes)



train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

#you have frozen the entire CLIP visual encoder:
# for param in clip_model.visual.parameters():
#     param.requires_grad = False

#unfreze all layers
for param in model.parameters():
    param.requires_grad = True

#unfreeze last 4 layers:
# for name, param in clip_model.visual.named_parameters():
#     if "transformer.resblocks.8" in name or \
#        "transformer.resblocks.9" in name or \
#        "transformer.resblocks.10" in name or \
#        "transformer.resblocks.11" in name:
#         param.requires_grad = True
#     else:
#         param.requires_grad = False


# class CLIPSkinClassifier(nn.Module):
#     def __init__(self, clip_model, num_classes=3):
#         super(CLIPSkinClassifier, self).__init__()
#         self.encoder = clip_model.visual
#         self.classifier = nn.Sequential(
#             nn.Linear(self.encoder.output_dim, 128),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(128, num_classes)
#         )
        

#     def forward(self, x):
#         features = self.encoder(x)
#         logits = self.classifier(features)
#         return logits



class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super(CLIPSkinClassifier, self).__init__()
        self.clip_model = clip_model
        self.fc1 = nn.Linear(clip_model.visual.output_dim, 512)  # Increased size
        self.fc2 = nn.Linear(512, 256)  # Adjusted for more capacity
        self.fc3 = nn.Linear(256, num_classes)
        self.batch_norm1 = nn.BatchNorm1d(512)
        self.batch_norm2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.clip_model.encode_image(x)
        x = torch.relu(self.fc1(x))
        x = self.batch_norm1(x)
        x = torch.relu(self.fc2(x))
        x = self.batch_norm2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x
    

def get_optimizer(optimizer_name, model_params, lr,weight_decay=0.01):
    if optimizer_name == 'AdamW':
        return AdamW(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'LAMB':
        return Lamb(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'SGD':
        return SGD(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'RAdam':
        return RAdam(model_params, lr=lr, weight_decay=weight_decay)


def compute_metrics(outputs, labels):
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)
    try:
        roc_auc = roc_auc_score(np.eye(3)[labels], F.softmax(outputs, dim=1).cpu().detach().numpy(), multi_class='ovr')
    except:
        roc_auc = None
    rmse = root_mean_squared_error(labels, preds)
    return acc, precision, recall, f1, roc_auc, rmse



def train_model(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_outputs = []
    all_labels = []

    for images, labels in tqdm(loader):
        images, labels = images.to(device).float(), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        all_outputs.append(outputs.detach())
        all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse



def evaluate_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(loader):
            images, labels = images.to(device).float(), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            all_outputs.append(outputs)
            all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse


# kfold = KFold(n_splits=k_folds, shuffle=True)

for batch_size in batch_sizes:
    for lr in learning_rates:
        for optimizer_name in optimizers_list:
            
            model = CLIPSkinClassifier(clip_model, num_classes=num_classes).to(device).float()
            
            
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
            
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
            
            
            criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
            optimizer = get_optimizer(optimizer_name,model.parameters(), lr=lr)
            
            
            for epoch in range(start,total_epochs,step):
                
                
                train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = train_model(model, train_loader, optimizer, criterion, device)
                val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse = evaluate_model(model, val_loader, criterion, device)
                test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = evaluate_model(model, test_loader, criterion, device)


                print("----------Values After Training-----------")
                print(f"\nEpoch: [{epoch}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}",
                    f"\nOptimizer: {optimizer_name}, \nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                        f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train  F1: {train_f1:.4f},"
                        f"Train ROC AUC: {train_roc_auc if train_roc_auc is not None else 'N/A'}, Train RMSE: {train_rmse:.4f}")

                print("\n\n-----------Values After Validation-----------")
                print(f"\nEpoch: [{epoch}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name}, \nVal Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                            f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc if train_roc_auc is not None else 'N/A'}, "
                            f"Val RMSE: {val_rmse:.4f}")

                print("\n\n-----------Values After Testing-----------")
                print(f"\nEpoch: [{epoch}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                    f"\nOptimizer: {optimizer_name}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Precision: {test_precision:.4f}, "
                            f"Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}, Test ROC AUC: {test_roc_auc if train_roc_auc is not None else 'N/A'}, Test RMSE: {test_rmse:.4f}")

                
                
                overall_result = {
                    'Epoch': epoch ,
                    'Batch Size': batch_size,
                    'Learning Rate': lr,
                    'Optimizer': optimizer_name,

                    'Train Loss': round(train_loss, 4),
                    'Test Loss': round(test_loss, 4),
                    'Val Loss': round(val_loss, 4),

                    'Train Acc': round(train_acc, 4),
                    'Test Acc': round(test_acc, 4),
                    'Val Acc': round(val_acc, 4),

                    'Train Precision': round(train_precision, 4),
                    'Test Precision': round(test_precision, 4),
                    'Val Precision': round(val_precision, 4),

                    'Train Recall': round(train_recall, 4),
                    'Test Recall': round(test_recall, 4),
                    'Val Recall': round(val_recall, 4),

                    'Train F1 Score': round(train_f1, 4),
                    'Test F1 Score': round(test_f1, 4),
                    'Val F1 Score': round(val_f1, 4),

                    'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
                    'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
                    'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,

                    'Train RMSE': round(train_rmse, 4),
                    'Test RMSE': round(test_rmse, 4),
                    'Val RMSE': round(val_rmse, 4)
                }

                overall_result_file = f'../Sample testing/Test 14/Batch {batch_size} model parameter testing.csv'
                if not os.path.isfile(overall_result_file):
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)
                else:
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)
                
                
                # model_save_path = f"../saved_models/Batch{batch_size}_LR{lr}_Optim{optimizer_name}.pth"
                # torch.save(model.state_dict(), model_save_path)
                # print(f"Model saved to {model_save_path}")



Clip based Vit with K folds

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
from torch_optimizer import Lamb  
import clip
from torch.optim import AdamW
from torch.optim import SGD
from pytorch_lamb import Lamb
from torch_optimizer import RAdam
from sklearn.model_selection import KFold
from torch.utils.data import Subset
from PIL import Image


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")



batch_sizes = [16]
#[4,8,16,32,64]
learning_rates = [0.001]
#[0.1,0.01,0.001,0.0001,0.00001]
optimizers_list = ['LAMB', 'RAdam']
#['LAMB', 'AdamW', 'SGD', 'RAdam']
num_classes=2
total_epochs = 501
start=100
step=5



clip_model, _ = clip.load("ViT-B/32", device=device)


#without augmentataion
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
                         (0.26862954, 0.26130258, 0.27577711))
])

#with augmentaion
# aug_transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(p=0.8),
#     transforms.RandomVerticalFlip(p=0.8),
#     transforms.RandomRotation(45),     
#     transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
#     #transforms.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(0.75, 1.25)),  
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize((0.48145466, 0.4578275, 0.40821073),
#                          (0.26862954, 0.26130258, 0.27577711))
# ])

data_dir = r"../Preprocessing/New Dataset"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

class RGBAImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = Image.open(path)
        
        # Convert palette images to RGBA to avoid warning
        if sample.mode == 'P':
            sample = Image.open(path).convert('RGB')
        
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target

# dataset = RGBAImageFolder(root=data_dir, transform=transform)
print("Classes:", dataset.classes)

train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

#you have frozen the entire CLIP visual encoder:
# for param in clip_model.visual.parameters():
#     param.requires_grad = False



#unfreeze last 4 layers:
# for name, param in clip_model.visual.named_parameters():
#     if "transformer.resblocks.8" in name or \
#        "transformer.resblocks.9" in name or \
#        "transformer.resblocks.10" in name or \
#        "transformer.resblocks.11" in name:
#         param.requires_grad = True
#     else:
#         param.requires_grad = False


# class CLIPSkinClassifier(nn.Module):
#     def __init__(self, clip_model, num_classes=3):
#         super(CLIPSkinClassifier, self).__init__()
#         self.encoder = clip_model.visual
#         self.classifier = nn.Sequential(
#             nn.Linear(self.encoder.output_dim, 128),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(128, num_classes)
#         )
        

#     def forward(self, x):
#         features = self.encoder(x)
#         logits = self.classifier(features)
#         return logits



class CLIPSkinClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super(CLIPSkinClassifier, self).__init__()
        self.clip_model = clip_model
        self.fc1 = nn.Linear(clip_model.visual.output_dim, 512)  # Increased size
        self.fc2 = nn.Linear(512, 256)  # Adjusted for more capacity
        self.fc3 = nn.Linear(256, num_classes)
        self.batch_norm1 = nn.BatchNorm1d(512)
        self.batch_norm2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.clip_model.encode_image(x)
        x = torch.relu(self.fc1(x))
        x = self.batch_norm1(x)
        x = torch.relu(self.fc2(x))
        x = self.batch_norm2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x
    

def get_optimizer(optimizer_name, model_params, lr,weight_decay=0.01):
    if optimizer_name == 'AdamW':
        return AdamW(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'LAMB':
        return Lamb(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'SGD':
        return SGD(model_params, lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'RAdam':
        return RAdam(model_params, lr=lr, weight_decay=weight_decay)


def compute_metrics(outputs, labels):
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    recall = recall_score(labels, preds, average='macro', zero_division=0)
    f1 = f1_score(labels, preds, average='macro', zero_division=0)
    try:
        roc_auc = roc_auc_score(np.eye(3)[labels], F.softmax(outputs, dim=1).cpu().detach().numpy(), multi_class='ovr')
    except:
        roc_auc = None
    rmse = root_mean_squared_error(labels, preds)
    return acc, precision, recall, f1, roc_auc, rmse



def train_model(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    all_outputs = []
    all_labels = []

    for images, labels in tqdm(loader):
        images, labels = images.to(device).float(), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        all_outputs.append(outputs.detach())
        all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse



def evaluate_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(loader):
            images, labels = images.to(device).float(), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            all_outputs.append(outputs)
            all_labels.append(labels)

    outputs_cat = torch.cat(all_outputs)
    labels_cat = torch.cat(all_labels)
    acc, precision, recall, f1, roc_auc, rmse = compute_metrics(outputs_cat, labels_cat)
    return total_loss / len(loader), acc, precision, recall, f1, roc_auc, rmse


# Define the number of folds
k_folds = 10

# Set up KFold
kfold = KFold(n_splits=k_folds, shuffle=True,random_state=42)

for batch_size in batch_sizes:
    for lr in learning_rates:
        for optimizer_name in optimizers_list:
            
            for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
                print(f'Fold {fold + 1}/{k_folds}')
                
                # Create subsets for training and validation
                train_subset = Subset(dataset, train_idx)
                val_subset = Subset(dataset, val_idx)
                
                # Create data loaders for training and validation
                train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
                val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
                
            
                model = CLIPSkinClassifier(clip_model, num_classes=num_classes).to(device).float()
                #unfreze all layers
                for param in model.parameters():
                    param.requires_grad = True
                
                
                # train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                
                # val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
                
                test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
                
                
                criterion = nn.CrossEntropyLoss(label_smoothing=0.2)
                optimizer = get_optimizer(optimizer_name,model.parameters(), lr=lr)
                
                
                for epoch in range(start,total_epochs,step):
                    
                    
                    train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = train_model(model, train_loader, optimizer, criterion, device)
                    val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse = evaluate_model(model, val_loader, criterion, device)
                    test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = evaluate_model(model, test_loader, criterion, device)


                    print("----------Values After Training-----------")
                    print(f"\nEpoch: [{epoch}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}",
                        f"\nOptimizer: {optimizer_name}, \nTrain Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                            f"Train Precision: {train_precision:.4f}, Train Recall: {train_recall:.4f}, Train  F1: {train_f1:.4f},"
                            f"Train ROC AUC: {train_roc_auc if train_roc_auc is not None else 'N/A'}, Train RMSE: {train_rmse:.4f}")

                    print("\n\n-----------Values After Validation-----------")
                    print(f"\nEpoch: [{epoch}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                        f"\nOptimizer: {optimizer_name}, \nVal Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                                f"Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc if train_roc_auc is not None else 'N/A'}, "
                                f"Val RMSE: {val_rmse:.4f}")

                    print("\n\n-----------Values After Testing-----------")
                    print(f"\nEpoch: [{epoch}/{total_epochs}] \nBatch Size: {batch_size} \nLearning Rate: {lr}"
                        f"\nOptimizer: {optimizer_name}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test Precision: {test_precision:.4f}, "
                                f"Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}, Test ROC AUC: {test_roc_auc if train_roc_auc is not None else 'N/A'}, Test RMSE: {test_rmse:.4f}")

                    
                    
                    overall_result = {
                        'Epoch': epoch ,
                        'Batch Size': batch_size,
                        'Learning Rate': lr,
                        'Optimizer': optimizer_name,

                        'Train Loss': round(train_loss, 4),
                        'Test Loss': round(test_loss, 4),
                        'Val Loss': round(val_loss, 4),

                        'Train Acc': round(train_acc, 4),
                        'Test Acc': round(test_acc, 4),
                        'Val Acc': round(val_acc, 4),

                        'Train Precision': round(train_precision, 4),
                        'Test Precision': round(test_precision, 4),
                        'Val Precision': round(val_precision, 4),

                        'Train Recall': round(train_recall, 4),
                        'Test Recall': round(test_recall, 4),
                        'Val Recall': round(val_recall, 4),

                        'Train F1 Score': round(train_f1, 4),
                        'Test F1 Score': round(test_f1, 4),
                        'Val F1 Score': round(val_f1, 4),

                        'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
                        'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
                        'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,

                        'Train RMSE': round(train_rmse, 4),
                        'Test RMSE': round(test_rmse, 4),
                        'Val RMSE': round(val_rmse, 4)
                    }

                    overall_result_file = f'../Sample testing/Test 15/Batch {batch_size} model parameter testing.csv'
                    if not os.path.isfile(overall_result_file):
                        pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)
                    else:
                        pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)
                    
                    
                    model_save_path = f"../Sample testing/Test 15/Batch{batch_size}_LR{lr}_Optim{optimizer_name}.pth"
                    torch.save(model.state_dict(), model_save_path)
                    print(f"Model saved to {model_save_path}")



In [ ]:
# model_save_path = f"../Sample testing/Test 15/Batch{batch_size}_LR{lr}_Optim{optimizer_name}.pth"
# torch.save(model.state_dict(), model_save_path)
# print(f"Model saved to {model_save_path}")


In [ ]:
import os

# Define the path to the directory
directory_path = '../Sample testing/'

# Count the number of subdirectories
dir_count = sum([1 for item in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, item))])

# Print the count of directories
print(f'Number of directories in {directory_path}: {dir_count}')

new_dir_path = f'../Sample testing/Test {dir_count+1}'

# Check if the directory exists, and create it if it does not
if not os.path.exists(new_dir_path):
    os.mkdir(new_dir_path)
    print(f'New directory created at {new_dir_path}')
else:
    print(f'Directory {new_dir_path} already exists.')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
import random
import pandas as pd
import os

# -------- 1. PARAMETERS --------
dataset_path = r'../Preprocessing/New Dataset'  # Folder with subfolders dry/, oily/, normal/
img_size = (128, 128)
batch_size = 32
epochs = 100
learning_rate = 0.001
seed = 42
num_classes = 3  # dry, oily, normal

# -------- 2. LOAD AND SPLIT DATA --------
# Load full dataset as unbatched individual images
full_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=img_size,
    label_mode='int',
    batch_size=None,  # For manual splitting
    shuffle=True,
    seed=seed,
    class_names=['dry', 'normal','oily']
)


# Convert dataset to list
data_list = list(full_dataset)
random.seed(seed)
random.shuffle(data_list)

# Manual 70/15/15 split
total_len = len(data_list)
train_len = int(0.7 * total_len)
val_len = int(0.15 * total_len)

train_data = data_list[:train_len]
val_data = data_list[train_len:train_len + val_len]
test_data = data_list[train_len + val_len:]

# Convert to tf.data.Dataset
def build_dataset(data_list, batch_size):
    images, labels = zip(*data_list)
    dataset = tf.data.Dataset.from_tensor_slices((list(images), list(labels)))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

train_ds = build_dataset(train_data, batch_size)
val_ds = build_dataset(val_data, batch_size)
test_ds = build_dataset(test_data, batch_size)

# -------- 3. BUILD MODEL (EfficientNetV2 + Custom Classifier) --------
base_model = EfficientNetV2B0(include_top=False, weights='imagenet', input_shape=img_size + (3,))
base_model.trainable = True  # UnFreeze base

with tf.device('/GPU:0'):
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

# -------- 4. TRAIN --------
model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=2)

# -------- 5. EVALUATE --------
train_loss, train_acc = model.evaluate(train_ds, verbose=2)
val_loss, val_acc = model.evaluate(val_ds, verbose=2)
test_loss, test_acc = model.evaluate(test_ds, verbose=2)


# -------- 6. SAVE MODEL --------
save_dir = '../saved_models/'
os.makedirs(save_dir, exist_ok=True)
model.save(os.path.join(save_dir, 'best_model.keras'))

# -------- 7. EXPORT RESULTS TO CSV --------
results = {
    'Train Loss': round(train_loss, 4),
    'Val Loss': round(val_loss, 4),
    'Test Loss': round(test_loss, 4),
    'Train Acc': round(train_acc, 4),
    'Val Acc': round(val_acc, 4),
    'Test Acc': round(test_acc, 4)
}

csv_file = f'../Sample testing/Test 15/Batch {batch_size} model parameter testing.csv'
os.makedirs(os.path.dirname(csv_file), exist_ok=True)

if not os.path.isfile(csv_file):
    pd.DataFrame([results]).to_csv(csv_file, index=False)
else:
    pd.DataFrame([results]).to_csv(csv_file, mode='a', index=False, header=False)

print("✅ Results saved to CSV.")
print("✅ Model saved to:", os.path.join(save_dir, 'best_model.keras'))
print(results)

# -------- 6. OUTPUT --------
# results = {
#     'Train Loss': round(train_loss, 4),
#     'Test Loss': round(test_loss, 4),
#     'Val Loss': round(val_loss, 4),



#     'Train Acc': round(train_acc, 4),
#     'Test Acc': round(test_acc, 4),
#     'Val Acc': round(val_acc, 4)
# }


# overall_result = {
#     'Epoch': epoch ,
#     'Batch Size': batch_size,
#     'Learning Rate': lr,
#     'Optimizer': optimizer_name,
#     'Train Loss': round(train_loss, 4),
#     'Test Loss': round(test_loss, 4),
#     'Val Loss': round(val_loss,4),
#     'Train Acc': round(train_acc, 4),
#     'Test Acc': round(test_acc, 4),
#     'Val Acc': round(val_acc, 4),
#     'Train Precision': round(train_precision, 4),
#     'Test Precision': round(test_precision, 4),
#     'Val Precision': round(val_precision, 4),
#     'Train Recall': round(train_recall, 4),
#     'Test Recall': round(test_recall, 4),
#     'Val Recall': round(val_recall, 4),
#     'Train F1 Score': round(train_f1, 4),
#     'Test F1 Score': round(test_f1, 4),
#     'Val F1 Score': round(val_f1, 4),
#     'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
#     'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
#     'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,
#     'Train RMSE': round(train_rmse, 4),
#     'Test RMSE': round(test_rmse, 4),
#     'Val RMSE': round(val_rmse, 4),
# }

# overall_result_file = f'../Sample testing/Test 15/Batch {batch_size} model parameter testing.csv'
# if not os.path.isfile(overall_result_file):
#     pd.DataFrame([results]).to_csv(overall_result_file, index=False)
# else:
#     pd.DataFrame([results]).to_csv(overall_result_file, mode='a', index=False, header=False)

# print(results)
# model = tf.keras.models.load_model("../saved_models/best_model.keras")

CNN with K folds

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
import pandas as pd
import numpy as np
import os
import random
from tensorflow.keras.optimizers import Adam, SGD
    
    
# -------- Parameters --------
img_size = (128, 128)
batch_size = 32
epochs = 50
learning_rate = 0.001
seed = 42
num_classes = 3
optimizer_name = "Adam"


# -------- Load Dataset as (image, label) pairs --------
dataset_path = r'../Preprocessing/New Dataset'
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=img_size,
    label_mode='int',
    batch_size=None,
    shuffle=True,
    seed=seed,
    class_names=['dry', 'normal','oily']
)
data_list = list(dataset)
random.shuffle(data_list)

# -------- Convert to numpy arrays --------
images, labels = zip(*data_list)
X = tf.stack(list(images))
y = np.array(labels)

# # -------- Create CSV if doesn't exist --------
# Initialize dictionary to hold the counts
class_names = ['dry', 'normal','oily']
class_counts = {}

# Iterate through each class (subfolder) and count the number of images
for class_name in class_names:
    class_path = os.path.join(dataset_path, class_name)
    # Count the number of image files (assuming image files are .jpg, .jpeg, .png)
    image_count = len([f for f in os.listdir(class_path) if f.endswith(('jpg', 'jpeg', 'png'))])
    class_counts[class_name] = image_count

# Display the class counts
print("Class distribution:")
for class_name, count in class_counts.items():
    print(f"{class_name}: {count} images")

# Calculate imbalance ratio (larger ratio means higher imbalance)
imbalance_ratio = max(class_counts.values()) / min(class_counts.values())
print(f"Imbalance Ratio: {imbalance_ratio}")

# -------- Cross-validation loop --------
kf = KFold(n_splits=10, shuffle=True, random_state=seed)

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    print(f"\n🌀 Fold {fold + 1}/10")

    X_train, X_test = tf.gather(X, train_idx), tf.gather(X, test_idx)
    y_train, y_test = y[train_idx], y[test_idx]

    val_split = int(0.15 * len(X_train))
    X_val, y_val = X_train[:val_split], y_train[:val_split]
    X_train, y_train = X_train[val_split:], y_train[val_split:]

    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    with tf.device('/GPU:0'):

        base_model = EfficientNetV2B0(include_top=False, weights='imagenet', input_shape=img_size + (3,))
        base_model.trainable = True

        model = Sequential([
            base_model,
            GlobalAveragePooling2D(),
            Dropout(0.5),
            Dense(128, activation='relu'),
            Dropout(0.5),
            Dense(num_classes, activation='softmax')
        ])

        optimizer = Adam(learning_rate=learning_rate, weight_decay=0.01)
        model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        
        class MetricsLogger(tf.keras.callbacks.Callback):
            def on_epoch_end(self, epoch, logs=None):
                def get_metrics(ds):
                    y_true = []
                    y_pred = []
                    for batch_x, batch_y in ds:
                        preds = model.predict(batch_x, verbose=0)
                        y_true.extend(batch_y.numpy())
                        y_pred.extend(np.argmax(preds, axis=1))
                    acc = np.mean(np.array(y_true) == np.array(y_pred))
                    precision = precision_score(y_true, y_pred, zero_division=0)
                    recall = recall_score(y_true, y_pred, zero_division=0)
                    f1 = f1_score(y_true, y_pred, zero_division=0)
                    roc_auc = roc_auc_score(y_true, y_pred) if len(set(y_true)) == 2 else None
                    rmse = root_mean_squared_error(y_true, y_pred)
                    loss = model.evaluate(ds, verbose=0)[0]
                    return loss, acc, precision, recall, f1, roc_auc, rmse

                train_loss, train_acc, train_precision, train_recall, train_f1, train_roc_auc, train_rmse = get_metrics(train_ds)
                val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse = get_metrics(val_ds)
                test_loss, test_acc, test_precision, test_recall, test_f1, test_roc_auc, test_rmse = get_metrics(test_ds)

                overall_result = {
                    'Epoch': epoch + 1,
                    'Batch Size': batch_size,
                    'Learning Rate': learning_rate,
                    'Optimizer': optimizer_name,
                    'Train Loss': round(train_loss, 4),
                    'Test Loss': round(test_loss, 4),
                    'Val Loss': round(val_loss, 4),
                    'Train Acc': round(train_acc, 4),
                    'Test Acc': round(test_acc, 4),
                    'Val Acc': round(val_acc, 4),
                    'Train Precision': round(train_precision, 4),
                    'Test Precision': round(test_precision, 4),
                    'Val Precision': round(val_precision, 4),
                    'Train Recall': round(train_recall, 4),
                    'Test Recall': round(test_recall, 4),
                    'Val Recall': round(val_recall, 4),
                    'Train F1 Score': round(train_f1, 4),
                    'Test F1 Score': round(test_f1, 4),
                    'Val F1 Score': round(val_f1, 4),
                    'Train ROC AUC': round(train_roc_auc, 4) if train_roc_auc is not None else None,
                    'Test ROC AUC': round(test_roc_auc, 4) if test_roc_auc is not None else None,
                    'Val ROC AUC': round(val_roc_auc, 4) if val_roc_auc is not None else None,
                    'Train RMSE': round(train_rmse, 4),
                    'Test RMSE': round(test_rmse, 4),
                    'Val RMSE': round(val_rmse, 4),
                }
                
                overall_result_file = f'../Sample testing/Test 15/Batch {batch_size} model parameter testing.csv'
                
                if not os.path.isfile(overall_result_file):
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, index=False)
                else:
                    pd.DataFrame([overall_result]).to_csv(overall_result_file, mode='a', index=False, header=False)

        # -------- Train model --------
        model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=2, callbacks=[MetricsLogger()])

    # Optional: save best model per fold
    model.save(f'../saved_models/model_fold_{fold + 1}.keras')


New Iqra Code

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow_addons.optimizers import AdamW

# === Paths ===
input_data_path = '../Preprocessing/New Dataset'
output_dir = '../Sample testing/Test 16/'
os.makedirs(output_dir, exist_ok=True)
csv_file = os.path.join(output_dir, 'final_averaged_results_all_batches.csv')

# === Hyperparameters ===
batch_sizes = [16]
learning_rates = [0.0001]
optimizers_list = ['Adam']
start, total_epochs, step, k = 100, 501, 5, 10

#batch_sizes = [8,16,32,64]
#learning_rates = [0.1 , 0.001 ,0.0001 , 0,00001]
#optimizers_list = ['LAMB', 'AdamW', 'SGD', 'RAdam']
# === Optimizer Selector ===
def get_optimizer(name, lr, wd=0.01):
    if name == 'Adam':
        return Adam(learning_rate=lr, weight_decay=wd)
    else:
        raise ValueError(f"Unsupported optimizer: {name}")

# === Load Image Paths and Labels ===
def load_data(image_dir):
    classes = sorted(os.listdir(image_dir))
    images, labels = [], []
    for label, cls in enumerate(classes):
        cls_folder = os.path.join(image_dir, cls)
        for fname in os.listdir(cls_folder):
            if fname.lower().endswith(('png', 'jpg', 'jpeg')):
                images.append(os.path.join(cls_folder, fname))
                labels.append(label)
    return np.array(images), np.array(labels), classes

# === Image Preprocessing ===
def preprocess_input(img_path, size=(224, 224)):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
    img = tf.keras.preprocessing.image.img_to_array(img)
    return img / 255.0

# === Build Model ===
def create_model(num_classes):
    base_model = EfficientNetV2B0(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(128, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=output)

# === Main Execution ===
image_paths, labels, class_names = load_data(input_data_path)

for batch_size in batch_sizes:
    for lr in learning_rates:
        for optimizer_name in optimizers_list:
            for epoch in range(start, total_epochs, step):
                kf = KFold(n_splits=k, shuffle=True, random_state=42)
                metrics = {'train': [], 'val': [], 'test': []}

                for train_index, test_val_index in kf.split(image_paths):
                    train_paths, test_val_paths = image_paths[train_index], image_paths[test_val_index]
                    train_labels, test_val_labels = labels[train_index], labels[test_val_index]
                    test_paths, val_paths, test_labels, val_labels = train_test_split(
                        test_val_paths, test_val_labels, test_size=0.5, random_state=42)

                    x_train = np.array([preprocess_input(p) for p in train_paths])
                    x_test = np.array([preprocess_input(p) for p in test_paths])
                    x_val = np.array([preprocess_input(p) for p in val_paths])

                    y_train = tf.keras.utils.to_categorical(train_labels, num_classes=len(class_names))
                    y_test = tf.keras.utils.to_categorical(test_labels, num_classes=len(class_names))
                    y_val = tf.keras.utils.to_categorical(val_labels, num_classes=len(class_names))

                    model = create_model(len(class_names))
                    optimizer = get_optimizer(optimizer_name, lr)
                    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
                    
                    
                    
                    model.fit(x_train, y_train, epochs=epoch, batch_size=batch_size, verbose=0)

                    def evaluate(x, y_true):
                        y_pred = model.predict(x, verbose=0)
                        y_true_cls = np.argmax(y_true, axis=1)
                        y_pred_cls = np.argmax(y_pred, axis=1)
                        return {
                            'loss': model.evaluate(x, y_true, verbose=0)[0],
                            'acc': accuracy_score(y_true_cls, y_pred_cls),
                            'precision': precision_score(y_true_cls, y_pred_cls, average='weighted'),
                            'recall': recall_score(y_true_cls, y_pred_cls, average='weighted'),
                            'f1': f1_score(y_true_cls, y_pred_cls, average='weighted'),
                            'roc_auc': roc_auc_score(y_true, y_pred, multi_class='ovr'),
                            'rmse': mean_squared_error(y_true, y_pred, squared=False)
                        }

                    metrics['train'].append(evaluate(x_train, y_train))
                    metrics['test'].append(evaluate(x_test, y_test))
                    metrics['val'].append(evaluate(x_val, y_val))

                def average_metrics(metric_list):
                    return {key: round(np.mean([m[key] for m in metric_list]), 4) for key in metric_list[0]}

                avg_train = average_metrics(metrics['train'])
                avg_test = average_metrics(metrics['test'])
                avg_val = average_metrics(metrics['val'])

                result = {
                    'Epoch': epoch, 'Batch Size': batch_size, 'Learning Rate': lr, 'Optimizer': optimizer_name,
                    'Train Loss': avg_train['loss'], 'Test Loss': avg_test['loss'], 'Val Loss': avg_val['loss'],
                    'Train Acc': avg_train['acc'], 'Test Acc': avg_test['acc'], 'Val Acc': avg_val['acc'],
                    'Train Precision': avg_train['precision'], 'Test Precision': avg_test['precision'], 'Val Precision': avg_val['precision'],
                    'Train Recall': avg_train['recall'], 'Test Recall': avg_test['recall'], 'Val Recall': avg_val['recall'],
                    'Train F1 Score': avg_train['f1'], 'Test F1 Score': avg_test['f1'], 'Val F1 Score': avg_val['f1'],
                    'Train ROC AUC': avg_train['roc_auc'], 'Test ROC AUC': avg_test['roc_auc'], 'Val ROC AUC': avg_val['roc_auc'],
                    'Train RMSE': avg_train['rmse'], 'Test RMSE': avg_test['rmse'], 'Val RMSE': avg_val['rmse']
                }

                pd.DataFrame([result]).to_csv(csv_file, mode='a', header=not os.path.exists(csv_file), index=False)